In [111]:
%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [115]:
%pylab inline
import gym
import pywren
import pywren.wrenconfig as wc
from gym import wrappers
import numpywren as npyw
import numpywren
from numpywren import matrix, matrix_utils
import scipy
import scipy.sparse
from sklearn.linear_model import SGDClassifier
from sklearn.kernel_approximation import RBFSampler
from sklearn import metrics 
from sklearn.svm import LinearSVC
from importlib import reload
from skimage.feature import blob_dog
from skimage.feature import canny
from scipy.sparse import csr_matrix
from scipy.sparse import hstack
from scipy.sparse import vstack

Populating the interactive namespace from numpy and matplotlib


ImportError: cannot import name 'matrix'

#  Blob Prost

In [ ]:
def canny_blob(state, x_ways, y_ways, with_blobs=False):
    """Run blob detection on canny edges. Optionally compute pairwise distances."""
    edges = canny(state)
    h, w = state.shape[0], state.shape[1]
    blobs = blob_dog(edges, max_sigma=5, threshold=0.2)
    nh, nw = int(np.ceil(h / y_ways)), int(np.ceil(w / x_ways))
    y, x, r = blobs[:, 0] // y_ways, blobs[:, 1] // x_ways, blobs[:, 2] * np.sqrt(2)  # downsize blobs
    y, x = y.astype(int), x.astype(int)
    featurized = np.zeros((nh, nw))
    featurized[y, x] = r  # fill in lower-dimensional representation
    if with_blobs:
        return csr_matrix(featurized.ravel()), np.vstack((y, x)).T
    return csr_matrix(featurized.ravel())

In [ ]:
def blob(frame, x_ways=8, y_ways=7, bins_per_color=5, with_prost=False):
    """Blob features for a single sample.

    :param frame: a single frame, hxwx3
    :param x_ways: Divide x dimension of sample by `x_ways`
    :param y_ways: Divide y dimension of sample by `y_ways`
    :param bins_per_color: Split each color into this many bins.
    :param with_prost: Add prost features
    :return: h2xw2x3 where h2=h/y_ways, w2=w/x_ways
    """
    all_features = []
    h, w = frame.shape[1: 3]
    bin_size = int(np.floor(255. / bins_per_color))
    nh, nw = int(np.ceil(h / y_ways)), int(np.ceil(w / x_ways))

    all_blobs = []
    for channel in range(frame.shape[2]):  # each channel for samples
      state = frame[:, :, channel]
      for bin_idx in range(bins_per_color):  # split each channel into multiple bins, evenly
        start, end = bin_idx * bin_size, (bin_idx + 1) * bin_size
        binned_state = np.zeros(state.shape)
        idxs = np.where(np.logical_and((state >= start), (state < end)))
        binned_state[idxs] = state[idxs]

        features, blobs = canny_blob(binned_state, x_ways=x_ways, y_ways=y_ways,  with_blobs=True)  # only look at values in this bin
        color = np.array([[channel * bins_per_color + bin_idx] * blobs.shape[0]]).T
        all_blobs.append(np.hstack((blobs[:, :2], color)))
        all_features.append(features)
    return hstack(all_features)

# Evaluation

In [ ]:
def agent_eval(agent, max_steps = float('inf'), game='SpaceInvaders-v0', seed=0, D=5):
    env = wrappers.Monitor(gym.make(game), "/tmp/atari/{0}".format(seed), force=True)
    env.seed(seed)
    state = blob(env.reset(), bins_per_color=D, with_prost=False)
    total_reward = 0
    i = 0
    done = False
    np.random.seed(seed)
    states = [state] * 4
    while ((not done) and i < max_steps):
        state, reward, done, info = env.step(agent(env, states))
        total_reward += reward
        i += 1
        states.pop(0)
        states.append(blob(state, bins_per_color=D, with_prost=False))
    return state, sum(env.get_episode_rewards())

In [ ]:
conf = wc.default()

In [ ]:
conf['runtime']['s3_bucket'] = "pictureweb"
conf['runtime']['s3_key'] = "pywren.runtime/pywren_runtime-3.6-rl.tar.gz"
pwex_standalone = pywren.standalone_executor(config=conf, job_max_runtime=600)

In [116]:
w = np.load('models/w_68_blob.npy')

In [92]:
def ls_agent(env, states):
    state = scipy.sparse.hstack(states)
    return np.argmax(state.dot(w))

In [93]:
def random_agent(env, _):
    return np.random.choice(env.action_space.n)

In [94]:
agent_eval(ls_agent)

(array([[[ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0],
         ..., 
         [ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0]],
 
        [[ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0],
         ..., 
         [ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0]],
 
        [[ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0],
         ..., 
         [ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0]],
 
        ..., 
        [[80, 89, 22],
         [80, 89, 22],
         [80, 89, 22],
         ..., 
         [80, 89, 22],
         [80, 89, 22],
         [80, 89, 22]],
 
        [[80, 89, 22],
         [80, 89, 22],
         [80, 89, 22],
         ..., 
         [80, 89, 22],
         [80, 89, 22],
         [80, 89, 22]],
 
        [[80, 89, 22],
         [80, 89, 22],
         [80, 89, 22],
         ..., 
         [80, 89, 22],
         [80, 89, 22],
         [80, 89, 22]]], dtype=uint8), 215.0)

In [ ]:
futures = pwex_standalone.map(lambda x: agent_eval(ls_agent, seed=int(x), D=5), range(200))

In [ ]:
%time pywren.wait(futures)

In [ ]:
rewards_ls = [f.result()[1] for f in futures]

In [ ]:
with open('models/w_68_blob_results.txt', 'w') as f:
    for reward in rewards_ls:
        f.write('%d\n' % int(reward))

In [ ]:
plt.title('36k Blob Features, Trained with 68 Episodes')
plt.hist(rewards_ls, bins=100)

In [103]:
print(np.mean(rewards_ls))
print(np.std(rewards_ls))

200.075
130.321216135


In [108]:
X_train_bigm = matrix.BigMatrix("space_invader_states_train", shape=[100*1024, 210, 160, 3], shard_sizes=[1024, 210, 160, 3], dtype="uint8", bucket="vaishaalpywren")
y_train_bigm = matrix.BigMatrix("space_invader_actions_rewards_train", shape=[100*1024, 2],  shard_sizes=[1024, 2], dtype="uint8", bucket="vaishaalpywren")

AttributeError: type object 'matrix' has no attribute 'BigMatrix'

In [ ]:
X_train_bigm.get_block(0,0,0,0)

In [104]:
import time
def blob_batch(X_train_bigm, y_train_bigm, block_idx, *blob_args, **blob_kwargs):
    states = X_train_bigm.get_block(block_idx,0,0,0)
    labels = y_train_bigm.get_block(block_idx,0,0,0)
    print("Downloaded now featurizing")
    all_features = []
    for i in range(states.shape[0][::10][:1000]):
        s = time.time()
        all_features.append(blob(states[i], *blob_args, **blob_kwargs))
        e = time.time()
    X = vstack(all_features)
    Y_oh = np.eye(6)[labels.ravel()]
    XTX = X.T.dot(X)
    XTY = X.T.dot(Y_oh)
    return XTX, XTY

def blob_batch_with_kernel (X_train_bigm, block_idx, kernel, *blob_args, **blob_kwargs):
    states = X_train_bigm.get_block(block_idx,0,0,0)
    print("Downloaded now featurizing")
    all_features = []
    for i in range(states.shape[0]):
        s = time.time()
        all_features.append(blobprost.blob(states[i], *blob_args, **blob_kwargs))
        e = time.time()
    return kernel.transform(np.vstack(all_features))

In [105]:
futures = pwex_standalone.map(lambda x: blob_batch(X_train_bigm, y_train_bigm, x), range(200))

In [106]:
%time pywren.wait(futures)

CPU times: user 3.22 s, sys: 254 ms, total: 3.48 s
Wall time: 13.1 s


([<pywren.future.ResponseFuture at 0x11126f438>,
 [])

In [107]:
matrices = [f.result()[1] for f in futures]

NameError: name 'X_train_bigm' is not defined

In [ ]:
kernel_approx.transform()

In [ ]:
reload(blobprost)
imshow(state_0[0])
plt.figure()
X_features = blob_batch(X_train_bigm, 0, bins_per_color=1, with_prost=False)

In [ ]:
kernel_approx = RBFSampler(gamma=1e-3, n_components=8192).fit(X_features)

In [ ]:
%time X_features = blob_batch_with_kernel(X_train_bigj, 0, kernel_approx, bins_per_color=1)

In [ ]:
kernel_approx

In [ ]:
futures = pwex_standalone.map(lambda x: blob_batch(X_train_bigm, x, bins_per_color=1), X_train_bigm._block_idxs(0))
%time pywren.wait(futures)
featurized = scipy.sparse.vstack([f.result() for f in futures])

In [ ]:
y_train = y_train_bigm.numpy()

In [ ]:
%time y_train_enc = np.eye(np.max(y_train[:, 0])+1)[y_train[:, 0]]

In [ ]:
clf_agent = SGDClassifier(verbose=3, n_iter=400, n_jobs=10)
clf_agent.fit(featurized, y_train[:, 0])

In [ ]:
metrics.accuracy_score(clf_agent.predict(featurized), y_train[:, 0])

In [ ]:
def _blob_agent(env, state, *blob_args, **blob_kwargs):
    return clf_agent.predict(blobprost.blob(state, *blob_args, **blob_kwargs))

In [ ]:
blob_agent = lambda x,y: _blob_agent(x, y, bins_per_color=1)

In [ ]:
futures = pwex_standalone.map(lambda x: agent_eval(blob_agent, seed=int(x)), range(150))

In [ ]:
rewards_clf = [f.result()[1] for f in futures]